In [1]:
#Stage 1
import json
import pandas as pd
import numpy as np
import joblib
import os
import re
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Load configuration
with open('config.json') as f:
    CONFIG = json.load(f)

# Create dataset-specific folder name
dataset_slug = re.sub(r'\W+', '_', os.path.splitext(os.path.basename(CONFIG["dataset_path"]))[0])

# Define dataset-specific output directories
base_dirs = {
    "predictions": os.path.join(CONFIG["predictions_dir"], dataset_slug),
    "models": os.path.join("models", dataset_slug),
    "features": os.path.join("features", dataset_slug),
    "metrics": os.path.join("metrics", dataset_slug),
    "hyperparams": os.path.join("hyperparams", dataset_slug),
    "fold_metrics": os.path.join("metrics", dataset_slug, "fold_wise")  # NEW
}

# Create all output directories
for path in base_dirs.values():
    os.makedirs(path, exist_ok=True)

def load_data():
    data = pd.read_csv(CONFIG["dataset_path"])
    X = data.drop("Target", axis=1)
    y = data["Target"]
    return train_test_split(X, y,
                            test_size=CONFIG["test_size"],
                            random_state=CONFIG["random_state"])

def get_feature_rankings(X_train, y_train):
    rankings = {}

    rf = RandomForestRegressor(n_estimators=200, max_depth=15, min_samples_split=5,
                                random_state=CONFIG["random_state"])
    rf.fit(X_train, y_train)
    rf_scores = rf.feature_importances_
    rankings['RF'] = list(X_train.columns[np.argsort(rf_scores)[::-1]])

    rfe = RFE(estimator=RandomForestRegressor(random_state=CONFIG["random_state"]),
              n_features_to_select=1)
    rfe.fit(X_train, y_train)
    rfe_scores = rfe.ranking_
    rankings['RFE_RF'] = list(X_train.columns[np.argsort(rfe_scores)])

    xgb = XGBRegressor(n_estimators=300, max_depth=5, learning_rate=0.1,
                       random_state=CONFIG["random_state"])
    xgb.fit(X_train, y_train)
    xgb_scores = xgb.feature_importances_
    rankings['XGB'] = list(X_train.columns[np.argsort(xgb_scores)[::-1]])

    hybrid_scores = (rf_scores + xgb_scores) / 2
    rankings['RF_XGB'] = list(X_train.columns[np.argsort(hybrid_scores)[::-1]])

    return rankings

def get_model_config(model_name):
    if model_name == 'RF':
        model_class = RandomForestRegressor
        param_grid = {
            'n_estimators': [100, 200, 300],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'random_state': [CONFIG["random_state"]]
        }
    elif model_name == 'XGB':
        model_class = XGBRegressor
        param_grid = {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.2],
            'subsample': [0.8, 1.0],
            'colsample_bytree': [0.8, 1.0],
            'random_state': [CONFIG["random_state"]]
        }
    return model_class, param_grid

def tune_hyperparameters(model, param_grid, X, y):
    scoring = {
        'r2': 'r2',
        'mae': 'neg_mean_absolute_error',
        'mse': 'neg_mean_squared_error'
    }
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring=scoring,
        cv=CONFIG["cv_folds"],
        n_jobs=-1,
        verbose=0,
        refit="r2"
    )
    grid_search.fit(X, y)
    ind = np.argmax(grid_search.cv_results_["mean_test_r2"])
    r2 = grid_search.cv_results_["mean_test_r2"][ind]
    mae = -1 * grid_search.cv_results_['mean_test_mae'][ind]
    mse = -1 * grid_search.cv_results_["mean_test_mse"][ind]
    return {
        "best_model": grid_search.best_estimator_,
        "best_params": grid_search.best_params_,
        "r2": r2,
        "mse": mse,
        "mae": mae
    }

def run_pipeline():
    X_train, X_test, y_train, y_test = load_data()
    feature_rankings = get_feature_rankings(X_train, y_train)
    all_results = []
    all_fold_metrics = []  
    combined_preds = pd.DataFrame({'true': y_test.reset_index(drop=True)})
    cv_train_preds = pd.DataFrame({'true': y_train.reset_index(drop=True)})

    for fs_name, features in feature_rankings.items():
        for model_name, Model in [('RF', RandomForestRegressor), ('XGB', XGBRegressor)]:
            print(f"\nEvaluating {model_name} with {fs_name} feature selection...")

            best_k = None
            best_cv_r2 = -np.inf

            # Step 1: Find best k using CV
            for k in range(1, len(features) + 1):
                k_features = features[:k]
                fold_r2 = []

                for train_idx, val_idx in KFold(CONFIG["cv_folds"]).split(X_train):
                    model = Model(random_state=CONFIG["random_state"])
                    model.fit(X_train.iloc[train_idx][k_features], y_train.iloc[train_idx])
                    preds = model.predict(X_train.iloc[val_idx][k_features])
                    fold_r2.append(r2_score(y_train.iloc[val_idx], preds))

                avg_r2 = np.mean(fold_r2)

                if avg_r2 > best_cv_r2:
                    best_cv_r2 = avg_r2
                    best_k = k

            # Step 2: HPO for spaced subset sizes
            subset_k_start = CONFIG.get("subset_k_start", 5)
            subset_k_step = CONFIG.get("subset_k_step", 5)
            subset_ks = [k for k in range(subset_k_start, best_k + 1, subset_k_step)]
            if best_k not in subset_ks:
                subset_ks.append(best_k)

            for k in subset_ks:
                k_features = features[:k]
                fold_preds = np.zeros(len(y_train))
                fold_r2 = []
                fold_mse = []
                fold_mae = []
                
                # Track per-fold metrics
                for fold_num, (train_idx, val_idx) in enumerate(KFold(CONFIG["cv_folds"]).split(X_train)):
                    model = Model(random_state=CONFIG["random_state"])
                    model.fit(X_train.iloc[train_idx][k_features], y_train.iloc[train_idx])
                    preds = model.predict(X_train.iloc[val_idx][k_features])
                    fold_preds[val_idx] = preds
                    
                    # Calculate metrics for this fold
                    r2 = r2_score(y_train.iloc[val_idx], preds)
                    mse = mean_squared_error(y_train.iloc[val_idx], preds)
                    mae = mean_absolute_error(y_train.iloc[val_idx], preds)
                    
                    fold_r2.append(r2)
                    fold_mse.append(mse)
                    fold_mae.append(mae)
                    
                    # Store fold-level results
                    all_fold_metrics.append({
                        'FS_Method': fs_name,
                        'Model': model_name,
                        'Features_Used': k,
                        'Fold': fold_num + 1,
                        'R2': r2,
                        'MSE': mse,
                        'MAE': mae
                    })

                cv_r2 = np.mean(fold_r2)
                cv_mse = np.mean(fold_mse)
                cv_mae = np.mean(fold_mae)
                
                # Calculate variance/std
                cv_r2_std = np.std(fold_r2)
                cv_mse_std = np.std(fold_mse)
                cv_mae_std = np.std(fold_mae)

                model_class, param_grid = get_model_config(model_name)
                print(f"  → HPO for {model_name} with {fs_name} using top-{k} features")
                hpo_results = tune_hyperparameters(
                    model_class(random_state=CONFIG["random_state"]),
                    param_grid,
                    X_train[k_features],
                    y_train
                )

                test_preds = hpo_results["best_model"].predict(X_test[k_features])
                test_r2 = r2_score(y_test, test_preds)
                test_mse = mean_squared_error(y_test, test_preds)
                test_mae = mean_absolute_error(y_test, test_preds)

                model_id = f"{fs_name}_{model_name}_Top{k}"
                combined_preds[f'pred_{model_id}'] = test_preds
                cv_train_preds[f'pred_{model_id}'] = fold_preds

                joblib.dump(hpo_results["best_model"], os.path.join(base_dirs["models"], f"Model_{model_id}.pkl"))

                with open(os.path.join(base_dirs["features"], f"Features_{model_id}.json"), 'w') as f:
                    json.dump(k_features, f)

                with open(os.path.join(base_dirs["hyperparams"], f"BestParams_{model_id}.json"), 'w') as f:
                    json.dump(hpo_results["best_params"], f)

                all_results.append({
                    'FS_Method': fs_name,
                    'Model': model_name,
                    'Features_Used': k,
                    'Best_Params': hpo_results["best_params"],
                    'CV_R2': cv_r2,
                    'CV_R2_Std': cv_r2_std,  
                    'CV_MSE': cv_mse,
                    'CV_MSE_Std': cv_mse_std,  
                    'CV_MAE': cv_mae,
                    'CV_MAE_Std': cv_mae_std, 
                    'Test_R2': test_r2,
                    'Test_MSE': test_mse,
                    'Test_MAE': test_mae
                })

    metrics_df = pd.DataFrame(all_results)
    metrics_df['R2_Rank'] = metrics_df['Test_R2'].rank(ascending=False, method='min')
    metrics_df = metrics_df.sort_values('R2_Rank')
    metrics_df.to_csv(os.path.join(base_dirs["metrics"], CONFIG['results_csv_path']), index=False)
    
    # Save fold-wise metrics
    fold_metrics_df = pd.DataFrame(all_fold_metrics)
    fold_metrics_df.to_csv(os.path.join(base_dirs["fold_metrics"], "stage1_fold_metrics.csv"), index=False)
    print(f"\n Saved fold-wise metrics to: {base_dirs['fold_metrics']}/stage1_fold_metrics.csv")
    
    combined_preds.to_csv(os.path.join(base_dirs["predictions"], "all_predictions.csv"), index=False)
    cv_train_preds.to_csv(os.path.join(base_dirs["predictions"], "cv_train_predictions.csv"), index=False)

if __name__ == '__main__':
    run_pipeline()


Evaluating RF with RF feature selection...
  → HPO for RF with RF using top-10 features
  → HPO for RF with RF using top-70 features
  → HPO for RF with RF using top-103 features

Evaluating XGB with RF feature selection...
  → HPO for XGB with RF using top-10 features
  → HPO for XGB with RF using top-70 features
  → HPO for XGB with RF using top-72 features

Evaluating RF with RFE_RF feature selection...
  → HPO for RF with RFE_RF using top-10 features
  → HPO for RF with RFE_RF using top-70 features
  → HPO for RF with RFE_RF using top-74 features

Evaluating XGB with RFE_RF feature selection...
  → HPO for XGB with RFE_RF using top-10 features
  → HPO for XGB with RFE_RF using top-42 features

Evaluating RF with XGB feature selection...
  → HPO for RF with XGB using top-10 features
  → HPO for RF with XGB using top-70 features
  → HPO for RF with XGB using top-130 features
  → HPO for RF with XGB using top-185 features

Evaluating XGB with XGB feature selection...
  → HPO for XGB 

In [3]:
#Stage 2
import pandas as pd
import numpy as np
import json 
import random
import os
import re
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.feature_selection import RFE

# ---------------------
# Load config & setup output
# ---------------------
with open("config.json") as f:
    CONFIG = json.load(f)

dataset_slug = re.sub(r'\W+', '_', os.path.splitext(os.path.basename(CONFIG["dataset_path"]))[0])
output_dir = os.path.join("metrics", dataset_slug)
fold_metrics_dir = os.path.join(output_dir, "fold_wise")
os.makedirs(output_dir, exist_ok=True)
os.makedirs(fold_metrics_dir, exist_ok=True)

print("="*80)
print(f"STAGE 2: META-MODEL ENSEMBLE WITH CROSS-VALIDATION")
print(f"Dataset: {dataset_slug}")
print("="*80)

# ---------------------
# Load prediction data
# ---------------------
cv_df = pd.read_csv(f"{CONFIG['predictions_dir']}/{dataset_slug}/cv_train_predictions.csv")
test_df = pd.read_csv(f"{CONFIG['predictions_dir']}/{dataset_slug}/all_predictions.csv")
metrics_df = pd.read_csv(f"metrics/{dataset_slug}/{CONFIG['results_csv_path']}")

X_train = cv_df.drop(columns=["true"])
y_train = cv_df["true"]
X_test = test_df.drop(columns=["true"])
y_test = test_df["true"]
model_columns = X_train.columns.tolist()

print(f"\nLoaded Data:")
print(f"  Training samples: {len(X_train)}")
print(f"  Test samples: {len(X_test)}")
print(f"  Base models available: {len(model_columns)}")
print(f"  Stage 1 configurations: {len(metrics_df)}")

# ---------------------
# Feature Selection Methods
# ---------------------
def fs_rf(X, y, n):
    """Random Forest feature importance"""
    rf = RandomForestRegressor(random_state=42, n_jobs=-1)
    rf.fit(X, y)
    scores = rf.feature_importances_
    return X.columns[np.argsort(scores)[::-1][:n]]

def fs_rfe_rf(X, y, n):
    """Recursive Feature Elimination with Random Forest"""
    rfe = RFE(estimator=RandomForestRegressor(random_state=42, n_jobs=-1), 
              n_features_to_select=n)
    rfe.fit(X, y)
    return X.columns[rfe.support_]

def fs_xgb(X, y, n):
    """XGBoost feature importance"""
    xgb = XGBRegressor(random_state=42, n_jobs=-1)
    xgb.fit(X, y)
    scores = xgb.feature_importances_
    return X.columns[np.argsort(scores)[::-1][:n]]

def fs_rf_xgb(X, y, n):
    """Hybrid: Average of RF and XGB importances"""
    rf = RandomForestRegressor(random_state=42, n_jobs=-1)
    xgb = XGBRegressor(random_state=42, n_jobs=-1)
    rf.fit(X, y)
    xgb.fit(X, y)
    avg_scores = (rf.feature_importances_ + xgb.feature_importances_) / 2
    return X.columns[np.argsort(avg_scores)[::-1][:n]]

fs_methods = {
    "RF": fs_rf,
    "RFE_RF": fs_rfe_rf,
    "XGB": fs_xgb,
    "RF_XGB": fs_rf_xgb
}

# Meta-models
def get_meta_model(model_name):
    """Get a fresh instance of meta-model"""
    if model_name == "LinearRegression":
        return LinearRegression()
    elif model_name == "RandomForest":
        return RandomForestRegressor(random_state=CONFIG["random_state"], n_jobs=-1)
    elif model_name == "XGBoost":
        return XGBRegressor(random_state=CONFIG["random_state"], n_jobs=-1)

meta_model_names = ["LinearRegression"]  # Only train LR for Method 3

# ---------------------
# Configure Top-K values
# ---------------------
max_models = len(metrics_df)
step = CONFIG.get("stage2_topk_step", 10)  # Default to 10 if not in config
k_values = list(range(10, max_models, step))
if max_models not in k_values:
    k_values.append(max_models)

print(f"\nTop-K values to evaluate: {k_values}")

# Storage for results
best_results = []
all_fold_metrics = []

# Setup CV
kfold = KFold(n_splits=CONFIG["cv_folds"], shuffle=True, random_state=CONFIG["random_state"])

print(f"\nUsing {CONFIG['cv_folds']}-fold cross-validation")

# ---------------------
# Main Loop: Evaluate Methods
# ---------------------
print("\n" + "="*80)
print("EVALUATING ENSEMBLE METHODS")
print("="*80)

for k in k_values:
    print(f"\n{'='*80}")
    print(f"Top-K = {k} Models")
    print(f"{'='*80}")
    
    # Get top-k models based on CV_R2 from Stage 1
    top_k_models = metrics_df.sort_values("CV_R2", ascending=False).head(k)
    top_k_cols = [
        f"pred_{row['FS_Method']}_{row['Model']}_Top{int(row['Features_Used'])}"
        for _, row in top_k_models.iterrows()
    ]
    
    print(f"Selected {len(top_k_cols)} top models based on Stage 1 CV performance")

    # ---------------------
    # Method 1: Random Averaging with CV
    # ---------------------
    print(f"\n[Method 1] Random Averaging (k={k})")
    
    random.seed(CONFIG["random_state"])
    random_cols = random.sample(model_columns, min(k, len(model_columns)))
    
    fold_metrics_random = []
    for fold_num, (train_idx, val_idx) in enumerate(kfold.split(X_train)):
        y_pred_val = X_train.iloc[val_idx][random_cols].mean(axis=1)
        
        r2 = r2_score(y_train.iloc[val_idx], y_pred_val)
        mse = mean_squared_error(y_train.iloc[val_idx], y_pred_val)
        mae = mean_absolute_error(y_train.iloc[val_idx], y_pred_val)
        
        fold_metrics_random.append({'R2': r2, 'MSE': mse, 'MAE': mae})
        
        all_fold_metrics.append({
            'Method': 'RandomAverage',
            'Top_K': k,
            'Meta_Model': 'None',
            'FS_Method': 'None',
            'Best_n': k,
            'Fold': fold_num + 1,
            'R2': r2,
            'MSE': mse,
            'MAE': mae
        })
    
    cv_r2_random = np.mean([f['R2'] for f in fold_metrics_random])
    cv_r2_std_random = np.std([f['R2'] for f in fold_metrics_random])
    cv_mse_random = np.mean([f['MSE'] for f in fold_metrics_random])
    cv_mae_random = np.mean([f['MAE'] for f in fold_metrics_random])
    
    # Test set evaluation
    y_pred_test_random = X_test[random_cols].mean(axis=1)
    test_r2_random = r2_score(y_test, y_pred_test_random)
    test_mse_random = mean_squared_error(y_test, y_pred_test_random)
    test_mae_random = mean_absolute_error(y_test, y_pred_test_random)
    
    print(f"  CV R² = {cv_r2_random:.4f} (±{cv_r2_std_random:.4f})")
    print(f"  Test R² = {test_r2_random:.4f}")

    best_results.append({
        "Dataset": dataset_slug,
        "Method": "RandomAverage",
        "Top_K": k,
        "Meta_Model": "None",
        "FS_Method": "None",
        "Best_n": k,
        "CV_R2": cv_r2_random,
        "CV_R2_Std": cv_r2_std_random,
        "CV_MSE": cv_mse_random,
        "CV_MSE_Std": np.std([f['MSE'] for f in fold_metrics_random]),
        "CV_MAE": cv_mae_random,
        "CV_MAE_Std": np.std([f['MAE'] for f in fold_metrics_random]),
        "Test_R2": test_r2_random,
        "Test_MSE": test_mse_random,
        "Test_MAE": test_mae_random,
        "Selected_Features": ','.join(random_cols[:5]) + '...' if len(random_cols) > 5 else ','.join(random_cols)
    })

    # ---------------------
    # Method 2: Top-K Averaging with CV
    # ---------------------
    print(f"\n[Method 2] Top-K Averaging (k={k})")
    
    fold_metrics_top = []
    for fold_num, (train_idx, val_idx) in enumerate(kfold.split(X_train)):
        y_pred_val = X_train.iloc[val_idx][top_k_cols].mean(axis=1)
        
        r2 = r2_score(y_train.iloc[val_idx], y_pred_val)
        mse = mean_squared_error(y_train.iloc[val_idx], y_pred_val)
        mae = mean_absolute_error(y_train.iloc[val_idx], y_pred_val)
        
        fold_metrics_top.append({'R2': r2, 'MSE': mse, 'MAE': mae})
        
        all_fold_metrics.append({
            'Method': 'TopKAverage',
            'Top_K': k,
            'Meta_Model': 'None',
            'FS_Method': 'None',
            'Best_n': k,
            'Fold': fold_num + 1,
            'R2': r2,
            'MSE': mse,
            'MAE': mae
        })
    
    cv_r2_top = np.mean([f['R2'] for f in fold_metrics_top])
    cv_r2_std_top = np.std([f['R2'] for f in fold_metrics_top])
    cv_mse_top = np.mean([f['MSE'] for f in fold_metrics_top])
    cv_mae_top = np.mean([f['MAE'] for f in fold_metrics_top])
    
    # Test set evaluation
    y_pred_test_top = X_test[top_k_cols].mean(axis=1)
    test_r2_top = r2_score(y_test, y_pred_test_top)
    test_mse_top = mean_squared_error(y_test, y_pred_test_top)
    test_mae_top = mean_absolute_error(y_test, y_pred_test_top)
    
    print(f"  CV R² = {cv_r2_top:.4f} (±{cv_r2_std_top:.4f})")
    print(f"  Test R² = {test_r2_top:.4f}")

    best_results.append({
        "Dataset": dataset_slug,
        "Method": "TopKAverage",
        "Top_K": k,
        "Meta_Model": "None",
        "FS_Method": "None",
        "Best_n": k,
        "CV_R2": cv_r2_top,
        "CV_R2_Std": cv_r2_std_top,
        "CV_MSE": cv_mse_top,
        "CV_MSE_Std": np.std([f['MSE'] for f in fold_metrics_top]),
        "CV_MAE": cv_mae_top,
        "CV_MAE_Std": np.std([f['MAE'] for f in fold_metrics_top]),
        "Test_R2": test_r2_top,
        "Test_MSE": test_mse_top,
        "Test_MAE": test_mae_top,
        "Selected_Features": ','.join(top_k_cols[:5]) + '...' if len(top_k_cols) > 5 else ','.join(top_k_cols)
    })

    # ---------------------
    # Method 3: FS + Linear Regression Meta-Model with Nested CV
    # ---------------------
    print(f"\n[Method 3] Feature Selection + Linear Regression (k={k})")
    
    X_train_top = X_train[top_k_cols]
    X_test_top = X_test[top_k_cols]

    for fs_name, fs_func in fs_methods.items():
        print(f"\n  Evaluating: {fs_name} + LinearRegression")
        
        best_r2 = -np.inf
        best_n = None
        best_metrics = {}
        best_fold_results = []

        # Try different numbers of features
        for n in range(2, min(k + 1, len(top_k_cols) + 1)):
            # Nested CV: outer loop for evaluation
            fold_results = []
            
            for fold_num, (train_idx, val_idx) in enumerate(kfold.split(X_train_top)):
                X_fold_train = X_train_top.iloc[train_idx]
                y_fold_train = y_train.iloc[train_idx]
                X_fold_val = X_train_top.iloc[val_idx]
                y_fold_val = y_train.iloc[val_idx]
                
                try:
                    # Feature selection on training fold
                    selected_feats = fs_func(X_fold_train, y_fold_train, n=n)
                    X_fold_train_fs = X_fold_train[selected_feats]
                    X_fold_val_fs = X_fold_val[selected_feats]
                    
                    # Train Linear Regression meta-model
                    model = LinearRegression()
                    model.fit(X_fold_train_fs, y_fold_train)
                    y_pred_val = model.predict(X_fold_val_fs)
                    
                    fold_results.append({
                        'R2': r2_score(y_fold_val, y_pred_val),
                        'MSE': mean_squared_error(y_fold_val, y_pred_val),
                        'MAE': mean_absolute_error(y_fold_val, y_pred_val),
                        'Selected_Features': list(selected_feats)
                    })
                except Exception as e:
                    print(f"    Warning: Failed for n={n}, fold={fold_num}: {e}")
                    continue
            
            if len(fold_results) == CONFIG["cv_folds"]:
                avg_r2 = np.mean([f['R2'] for f in fold_results])
                
                if avg_r2 > best_r2:
                    best_r2 = avg_r2
                    best_n = n
                    best_fold_results = fold_results
                    best_metrics = {
                        "CV_R2": avg_r2,
                        "CV_R2_Std": np.std([f['R2'] for f in fold_results]),
                        "CV_MSE": np.mean([f['MSE'] for f in fold_results]),
                        "CV_MSE_Std": np.std([f['MSE'] for f in fold_results]),
                        "CV_MAE": np.mean([f['MAE'] for f in fold_results]),
                        "CV_MAE_Std": np.std([f['MAE'] for f in fold_results]),
                    }
        
        if best_n is None:
            print(f"    ⚠ Skipped: No valid configuration found")
            continue
        
        # Store fold-wise results for best n
        for fold_num, fold_result in enumerate(best_fold_results):
            all_fold_metrics.append({
                'Method': 'FS_MetaModel',
                'Top_K': k,
                'Meta_Model': 'LinearRegression',
                'FS_Method': fs_name,
                'Best_n': best_n,
                'Fold': fold_num + 1,
                'R2': fold_result['R2'],
                'MSE': fold_result['MSE'],
                'MAE': fold_result['MAE']
            })
        
        # Final test evaluation with best n
        try:
            selected_feats = fs_func(X_train_top, y_train, n=best_n)
            X_train_fs = X_train_top[selected_feats]
            X_test_fs = X_test_top[selected_feats]
            
            final_model = LinearRegression()
            final_model.fit(X_train_fs, y_train)
            y_pred_test = final_model.predict(X_test_fs)
            
            test_r2 = r2_score(y_test, y_pred_test)
            test_mse = mean_squared_error(y_test, y_pred_test)
            test_mae = mean_absolute_error(y_test, y_pred_test)
            
            print(f"    CV R² = {best_metrics['CV_R2']:.4f} (±{best_metrics['CV_R2_Std']:.4f}), Test R² = {test_r2:.4f} (n={best_n})")
            
            best_results.append({
                "Dataset": dataset_slug,
                "Method": "FS_MetaModel",
                "Top_K": k,
                "Meta_Model": "LinearRegression",
                "FS_Method": fs_name,
                "Best_n": best_n,
                **best_metrics,
                "Test_R2": test_r2,
                "Test_MSE": test_mse,
                "Test_MAE": test_mae,
                "Selected_Features": ','.join(list(selected_feats)[:5]) + '...' if len(selected_feats) > 5 else ','.join(list(selected_feats))
            })
        except Exception as e:
            print(f"     Failed final evaluation: {e}")

# ---------------------
# Save Results
# ---------------------
print("\n" + "="*80)
print("SAVING RESULTS")
print("="*80)

# Save aggregated results
results_df = pd.DataFrame(best_results)
results_df = results_df.sort_values('Test_R2', ascending=False)
results_path = os.path.join(output_dir, "best_meta_model_results.csv")
results_df.to_csv(results_path, index=False)
print(f" Saved aggregated results: {results_path}")

# Save fold-wise metrics
fold_metrics_df = pd.DataFrame(all_fold_metrics)
fold_metrics_path = os.path.join(fold_metrics_dir, "stage2_fold_metrics.csv")
fold_metrics_df.to_csv(fold_metrics_path, index=False)
print(f" Saved fold-wise metrics: {fold_metrics_path}")

# Print summary
print("\n" + "="*80)
print("SUMMARY")
print("="*80)

print(f"\nTotal configurations evaluated: {len(results_df)}")
print(f"\nTop 5 Configurations by Test R²:")
print(results_df[['Method', 'Top_K', 'Meta_Model', 'FS_Method', 'Best_n', 'CV_R2', 'CV_R2_Std', 'Test_R2']].head(10).to_string(index=False))

print("\n" + "="*80)
print("STAGE 2 COMPLETE!")
print("="*80)

STAGE 2: META-MODEL ENSEMBLE WITH CROSS-VALIDATION
Dataset: PRF_LiDAR_702

Loaded Data:
  Training samples: 224
  Test samples: 25
  Base models available: 22
  Stage 1 configurations: 22

Top-K values to evaluate: [10, 20, 22]

Using 10-fold cross-validation

EVALUATING ENSEMBLE METHODS

Top-K = 10 Models
Selected 10 top models based on Stage 1 CV performance

[Method 1] Random Averaging (k=10)
  CV R² = 0.7005 (±0.1487)
  Test R² = 0.8544

[Method 2] Top-K Averaging (k=10)
  CV R² = 0.7099 (±0.1505)
  Test R² = 0.8552

[Method 3] Feature Selection + Linear Regression (k=10)

  Evaluating: RF + LinearRegression
    CV R² = 0.7075 (±0.1214), Test R² = 0.8301 (n=2)

  Evaluating: RFE_RF + LinearRegression
    CV R² = 0.7051 (±0.1327), Test R² = 0.8094 (n=7)

  Evaluating: XGB + LinearRegression
    CV R² = 0.7023 (±0.1373), Test R² = 0.8540 (n=7)

  Evaluating: RF_XGB + LinearRegression
    CV R² = 0.7027 (±0.1340), Test R² = 0.8077 (n=8)

Top-K = 20 Models
Selected 20 top models based 